In [72]:
# import dependencies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# display multiple print results on one line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# expands .describe with more info about the data set
import pandas_profiling

# sqlalchemy dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func
from sqlalchemy import inspect

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Text

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# flask 
from flask import Flask, jsonify

height has been deprecated.



In [2]:
# Use SQLAlchemy create_engine to connect to database
engine = create_engine("sqlite:///bbb.sqlite")
conn = engine.connect()

In [4]:
# use SQLAlchemy automap_base() to reflect tables into classes and save references
# called sample_names, otu_des, sample_meta, frequency
Base = automap_base()

In [5]:
# use the base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [6]:
Base.classes.keys()

['otu', 'samples', 'samples_metadata']

In [10]:
# assign classes to variables
sample_names = Base.classes.samples
otu_des = Base.classes.otu
sample_meta = Base.classes.samples_metadata

In [11]:
# create a session
# a session means that you want to do something to your data
session = Session(engine)

In [33]:
inspector = inspect(engine)

In [35]:
# get table information
print (inspector.get_table_names())

['otu', 'samples', 'samples_metadata']


In [48]:
# get col names for samples and stores in a tuple within a list
sample_name_col = inspector.get_columns('samples')

In [61]:
# prints list of sample names
sample_names_list = [item["name"] for item in sample_name_col]
del sample_names_list[0]

In [63]:
# prints list of sample_names
# sample_names_list

In [68]:
# return list of OTU descriptions
otu_results = session.query(otu_des.lowest_taxonomic_unit_found).all()

# stores otu results as a list
otu_results_list = list(np.ravel(otu_results))

In [78]:
# read Belly_Button_Biodiversity_Metadata
bbb_meta_df = pd.read_csv('Belly_Button_Biodiversity_Metadata.csv')

In [79]:
bbb_meta_df.head()
bbb_meta_df.shape

,SAMPLEID,EVENT,ETHNICITY,GENDER,AGE,WFREQ,BBTYPE,LOCATION,COUNTRY012,ZIP012,COUNTRY1319,ZIP1319,DOG,CAT,IMPSURFACE013,NPP013,MMAXTEMP013,PFC013,IMPSURFACE1319,NPP1319,MMAXTEMP1319,PFC1319
0,940,BellyButtonsScienceOnline,Caucasian,F,24.0,2.0,I,Beaufort/NC,usa,22306,usa,22306,no,no,8852.0,37.172222,54.5,NaN,1,NaN,33.990002,25.5
1,941,NaN,Caucasian/Midleastern,F,34.0,1.0,I,Chicago/IL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,943,BellyButtonsScienceOnline,Caucasian,F,49.0,1.0,I,Omaha/NE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,944,BellyButtonsScienceOnline,European,M,44.0,1.0,I,NewHaven/CT,usa,7079,usa,8822,no,yes,NaN,35.816666,16.0,NaN,0,6567.0,32.403332,28.5
4,945,BellyButtonsScienceOnline,Caucasian,F,48.0,1.0,I,Philidelphia/PA,usa,84404,usa,96025,no,no,NaN,37.783333,4.0,NaN,0,5613.0,33.634445,24.0


(153, 22)

In [82]:
bbb_meta_json = bbb_meta_df.to_json
bbb_meta_json

<bound method NDFrame.to_json of      SAMPLEID                      EVENT              ETHNICITY GENDER   AGE  WFREQ BBTYPE         LOCATION      COUNTRY012         ZIP012 COUNTRY1319        ZIP1319  DOG  CAT  IMPSURFACE013     NPP013  MMAXTEMP013  PFC013 IMPSURFACE1319  NPP1319  MMAXTEMP1319  PFC1319
0         940  BellyButtonsScienceOnline              Caucasian      F  24.0    2.0      I      Beaufort/NC             usa          22306         usa          22306   no   no         8852.0  37.172222         54.5     NaN              1      NaN     33.990002     25.5
1         941                        NaN  Caucasian/Midleastern      F  34.0    1.0      I       Chicago/IL             NaN            NaN         NaN            NaN  NaN  NaN            NaN        NaN          NaN     NaN            NaN      NaN           NaN      NaN
2         943  BellyButtonsScienceOnline              Caucasian      F  49.0    1.0      I         Omaha/NE             NaN            NaN         NaN       